In [1]:
import pyro
import torch
import pyro.distributions as pdist
import torch.distributions as tdist
import arviz
import numpy as np


from torch.distributions import constraints


c:\Users\simon\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class MyDensity(pdist.TorchDistribution):
    # The integration interval
    support = constraints.interval(-3.0, 3.0)
    # Constraint for the starting value used in the sampling
    # (should be within integration interval)
    arg_constraints = {"start": support}

    def __init__(self, start=torch.tensor(0.0)):
      # start = starting value for HMC sampling, default 0
      self.start = start
      super(pdist.TorchDistribution, self).__init__()

    def sample(self, sample_shape=torch.Size()):
        # This is only used to start the HMC sampling
        # It simply returns the starting value for the sampling
        return self.start

    def log_prob(self, x):
        density = torch.exp(-x**2/2) * (torch.sin(x)**2+ 3 * torch.cos(x)**2 * torch.sin(7*x)**2 + 1)
        log_prob = torch.log(density)
        return log_prob


In [3]:
def model():
    x = pyro.sample("x", MyDensity())
    return x


In [ ]:
nuts_kernel = pyro.infer.NUTS(model)
mcmc = pyro.infer.MCMC(nuts_kernel, num_samples=100, warmup_steps=50)
mcmc.run()

Warmup [2]:   0%|          | 0/150 [01:45, ?it/s]


KeyboardInterrupt: 

In [ ]:
samples = mcmc.get_samples()["x"].detach().cpu().numpy()
#Calculate E[x^2]
expectation_x2 = np.mean(samples**2)
data = arviz.from_pyro(mcmc)
summary = arviz.summary(data)
def HMC(n_samples):
    nuts_kernel = pyro.infer.NUTS(model)
    mcmc = pyro.infer.MCMC(nuts_kernel, num_samples=n_samples, warmup_steps=200)
    mcmc.run()
    samples = mcmc.get_samples()["x"].detach().cpu().numpy()
    expected_val = np.mean(samples**2)
    return expected_val
result = HMC(1000)
print("E[x^2] =", result)